# R4T1

# Openning Data

In [ ]:
using SparseArrays
using JuMP
using Ipopt
using GLPK
using LinearAlgebra
using MAT

In [ ]:
file = matopen("R4T1.mat")
varnames = names(file)

In [ ]:
S = read(file, "S")
L = read(file, "L")
U = read(file, "U")
lambda = read(file,"lambda")

In [ ]:
(m,n)=size(S)
(r,c) = size(L)

# constructing model

In [ ]:
mzeros = zeros(m,1);

In [ ]:
function opt_vector0(S,l,u,z)
    model = Model(Ipopt.Optimizer)
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r))
    optimize!(model)
    
    return JuMP.value.(v)
end

In [ ]:
function opt_vector1(S,l,u,z)
    model = Model(Ipopt.Optimizer)
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @constraint(model,-10^-3*mzeros .<= S*v .<= 10^-3*mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r))
    optimize!(model)
    
    return JuMP.value.(v)
end

In [ ]:
function opt_vector2(S,l,u,z)
    model = Model(Ipopt.Optimizer)
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @variable(model,b[1:m])
    
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    
    @constraint(model, b >= S*v)
    @constraint(model, b >= S*v)
    
    
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r) + lambda*sum(b[j] for j=1:m))
    optimize!(model)
    
    return JuMP.value.(v)
end

In [ ]:
function zero_rows(V)
    indexes= zeros(Int64,size(V)[1],1)
    k=1
    for i in 1:size(V)[1]
        if sum(V[i,:].*V[i,:])<1e-10
            indexes[k]= i
            k+=1
        end
    end
    return length(indexes[1:k-1])
end

In [ ]:
function row_squares(V)
    r,c = size(V,1), size(V,2)
    squares = zeros(r,1)
    for i in 1:r
        z=0
        for j in 1:c
           z += (V[i,j])^2
        end
        #squares[i]=sqrt(z)
        squares[i]=z
    end
    return squares
end

In [ ]:
l = L[:,1]
u = U[:,1]
z = ones(r,1)
v = opt_vector2(S,l,u,z)
zero_rows(v)

In [ ]:
V=v;

In [ ]:
for i in 2:c
    if isdefined(Main, :IJulia)
        Main.IJulia.stdio_bytes[] = 0
    end
    l = L[:,i]
    u = U[:,i]
    z = row_squares(V)
    v = opt_vector2(S,l,u,z)
    V = [V v]
    print("**i : ",i,"\n")
    print("***",zero_rows(V),"\n") 
end

In [ ]:
V

In [ ]:
n1 = zero_rows(V)
print(n1,"\n")
n2 = zero_rows((S*V)')
print(n2,"\n")
tot = zero_rows(V)+lambda*zero_rows((S*V)')
print(tot,"\n")

# writing to output file

In [ ]:
using DelimitedFiles
open("output.txt","w") do io
    writedlm(io, V,",")
end